In [ ]:
# -----------------------------------------------------------
# 1) GEREKLİ KÜTÜPHANELERİN KURULUMU
# -----------------------------------------------------------
!pip install transformers datasets accelerate sentencepiece -q

# -----------------------------------------------------------
# 2) BASIT VE HIZLI EĞİTİM (CPU/GPU Hibrit)
# -----------------------------------------------------------
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer
)

# Bellek temizliği
if torch.cuda.is_available():
    torch.cuda.empty_cache()

MODEL_NAME = "HuggingFaceTB/SmolLM2-135M"  # 360M → 135M (çok daha küçük model)

print(f"🔧 Model: {MODEL_NAME}")
print(f"🖥️  Cihaz: {'GPU' if torch.cuda.is_available() else 'CPU'}")

# -----------------------------------------------------------
# 3) TOKENIZER
# -----------------------------------------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.padding_side = "right"  ##   aa  sağdkasdkas aa

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -----------------------------------------------------------
# 4) MODEL (Basit FP32 veya FP16)
# -----------------------------------------------------------
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.config.pad_token_id = tokenizer.pad_token_id

# GPU varsa modeli GPU'ya taşı
if torch.cuda.is_available():
    model = model.cuda()
    print("✅ Model GPU'ya yüklendi")
else:
    print("✅ Model CPU'da çalışıyor")

# -----------------------------------------------------------
# 5) DATASET (ÇOK KÜÇÜK - Hızlı test için)
# -----------------------------------------------------------
dataset = load_dataset("vicgalle/alpaca-gpt4", split="train[:50]")  # Sadece 50 örnek!
dataset = dataset.train_test_split(test_size=0.2)

print(f"📊 Eğitim: {len(dataset['train'])} | Test: {len(dataset['test'])} örnek")

# -----------------------------------------------------------
# 6) TOKENIZATION (Kısa metinler)
# -----------------------------------------------------------
def format_example(example):
    instruction = example["instruction"]
    response = example["output"][:100]  # Response'u kısalt
    
    prompt = f"### Instruction:\n{instruction}\n\n### Response:\n{response}"
    
    encoded = tokenizer(
        prompt,
        max_length=64,  # Çok kısa tokenlar
        padding="max_length",
        truncation=True,
        return_tensors=None
    )
    
    encoded["labels"] = encoded["input_ids"].copy()
    return encoded

dataset = dataset.map(
    format_example,
    remove_columns=dataset["train"].column_names
)

# -----------------------------------------------------------
# 7) DATA COLLATOR
# -----------------------------------------------------------
collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# -----------------------------------------------------------
# 8) EĞITIM PARAMETRELERI (Minimal)
# -----------------------------------------------------------
args = TrainingArguments(
    output_dir="smollm_mini_finetuned",
    per_device_train_batch_size=2,  # Küçük batch
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    num_train_epochs=2,  # 2 epoch
    fp16=torch.cuda.is_available(),  # GPU varsa FP16
    logging_steps=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    report_to="none",
    no_cuda=not torch.cuda.is_available(),  # CPU varsa no_cuda=True
)

# -----------------------------------------------------------
# 9) TRAINER
# -----------------------------------------------------------
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=collator,
)

# -----------------------------------------------------------
# 10) EĞITIM
# -----------------------------------------------------------
print("\n🚀 Eğitim başlıyor (50 örnek, 2 epoch)...")
trainer.train()

# -----------------------------------------------------------
# 11) KAYDETME
# -----------------------------------------------------------
trainer.save_model("smollm_mini_finetuned")
tokenizer.save_pretrained("smollm_mini_finetuned")

print("\n✅ Eğitim tamamlandı! Model 'smollm_mini_finetuned' klasörüne kaydedildi.")
print("📌 Bu çok küçük bir örnektir. Gerçek eğitim için daha fazla veri kullanın.")



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
🔧 Model: HuggingFaceTB/SmolLM2-135M
🖥️  Cihaz: GPU
🔧 Model: HuggingFaceTB/SmolLM2-135M
🖥️  Cihaz: GPU
✅ Model GPU'ya yüklendi
✅ Model GPU'ya yüklendi
📊 Eğitim: 40 | Test: 10 örnek
📊 Eğitim: 40 | Test: 10 örnek


Map: 100%|██████████| 10/10 [00:00<00:00, 1766.10 examples/s]


🚀 Eğitim başlıyor (50 örnek, 2 epoch)...


Epoch,Training Loss,Validation Loss
1,1.909700,1.735289
2,1.236100,1.692337



✅ Eğitim tamamlandı! Model 'smollm_mini_finetuned' klasörüne kaydedildi.
📌 Bu çok küçük bir örnektir. Gerçek eğitim için daha fazla veri kullanın.


In [3]:
from transformers import pipeline

# Eğitilmiş modeli yükle
pipe = pipeline("text-generation", model="smollm_mini_finetuned", tokenizer="smollm_mini_finetuned")

# Test prompts
test_prompts = [
    "Explain machine learning:",
    "What is AI?",
    "How do transformers work?"
]

print("🤖 Model Test Sonuçları:\n" + "="*80 + "\n")

for i, prompt in enumerate(test_prompts, 1):
    print(f"{i}. Prompt: {prompt}")
    result = pipe(prompt, max_length=100, num_return_sequences=1)[0]["generated_text"]
    print(f"   Cevap: {result}")
    print("-" * 80 + "\n")

print("✅ Test tamamlandı!")


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strat

🤖 Model Test Sonuçları:

1. Prompt: Explain machine learning:


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


   Cevap: Explain machine learning:

Machine learning algorithms are used to find patterns in data for a variety of tasks. With machine learning, we can use algorithms to find patterns in data. For example, machine learning algorithms can be used to predict the weather.

Machine learning is a branch of artificial intelligence (AI) that uses algorithms to find patterns in data. We can use machine learning algorithms to predict the weather.

Machine learning algorithms are used to find patterns in data.

Machine learning algorithms can be used to find patterns in data.

Machine learning algorithms can be used to find patterns in data.

Machine learning is a branch of artificial intelligence (AI). AI refers to the use of computers to think and act like humans. Machine learning is a type of AI that uses algorithms and data to find patterns in data.

Machine learning algorithms are used to find patterns in data.

Machine learning algorithms can be used to find patterns in data.

Machine lea

Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


   Cevap: What is AI?

Artificial Intelligence (AI) is a rapidly evolving field that uses computer systems to mimic human intelligence processes such as problem solving, decision making, and language translation. Over the past few decades, AI has been applied across various industries, including healthcare, finance, transportation, and manufacturing. However, its potential impacts on our daily lives and society are still largely unknown.

Section 1: Defining AI

To understand AI better, let's break down the term into simpler terms:

* Artificial: Artificial in reference to human-like creations.
* Intelligence: Ability to understand, process information, and adapt to new situations.
* Intelligence Machines: Devices that simulate human cognition.
* Intelligence Machines: Devices that perform tasks that normally require human intelligence.

In simple terms, AI refers to the ability of a computer system to perform tasks that typically require human intelligence. These tasks may include und